In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv2_imshow #this is used for imshow to work on google colab

In [ ]:
def remove_shadow(image, gaussian_kernel_size=99, sigma=50):
    """
    Remove shadows from an image using Gaussian filtering

    Parameters:
    image: Input image (grayscale or color)
    gaussian_kernel_size: Size of the Gaussian kernel (must be odd)
    sigma: Standard deviation of Gaussian kernel

    Returns:
    shadow_removed: Image with shadows reduced/removed
    """
    # Convert to float32 for processing
    img_float = image.astype(np.float32) / 255.0

    if len(image.shape) == 3:
        # Process each channel separately for color images
        shadow_removed_channels = []
        for channel in cv2.split(img_float):
            # Get the background illumination using a large Gaussian kernel
            background = cv2.GaussianBlur(channel,
                                        (gaussian_kernel_size, gaussian_kernel_size),
                                        sigma)

            # Remove the background illumination
            shadow_removed_channel = channel / (background + 1e-6)  # Add small value to avoid division by zero

            # Normalize to [0, 1] range
            shadow_removed_channel = (shadow_removed_channel - np.min(shadow_removed_channel)) / \
                                   (np.max(shadow_removed_channel) - np.min(shadow_removed_channel))

            shadow_removed_channels.append(shadow_removed_channel)

        # Merge channels back
        shadow_removed = cv2.merge(shadow_removed_channels)
    else:
        # Process grayscale image
        background = cv2.GaussianBlur(img_float,
                                    (gaussian_kernel_size, gaussian_kernel_size),
                                    sigma)

        # Remove the background illumination
        shadow_removed = img_float / (background + 1e-6)

        # Normalize to [0, 1] range
        shadow_removed = (shadow_removed - np.min(shadow_removed)) / \
                        (np.max(shadow_removed) - np.min(shadow_removed))

    # Convert back to uint8
    shadow_removed = (shadow_removed * 255).astype(np.uint8)

    return shadow_removed

def adjust_contrast_brightness(image, alpha=1.2, beta=10):
    """
    Adjust contrast and brightness of the image

    Parameters:
    image: Input image
    alpha: Contrast control (1.0-3.0)
    beta: Brightness control (0-100)

    Returns:
    adjusted: Contrast and brightness adjusted image
    """
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)



In [ ]:
# Load image
image = cv2.imread('input_image.jpg')

# Remove shadows
shadow_removed = remove_shadow(image,
                             gaussian_kernel_size=99,  # Should be odd and large enough
                             sigma=50)  # Adjust based on shadow size

# Optional: Adjust contrast and brightness if needed
final_result = adjust_contrast_brightness(shadow_removed,
                                        alpha=1.2,  # Increase for more contrast
                                        beta=10)    # Increase for more brightness

# Save or display results
cv2.imwrite('shadow_removed.jpg', shadow_removed)
cv2.imwrite('final_result.jpg', final_result)
